In [ ]:
from flask import Flask, render_template, jsonify
from pymongo import MongoClient
from datetime import datetime, timedelta
from apscheduler.schedulers.background import BackgroundScheduler
from flask import request


app = Flask(__name__, static_url_path='/static')

# MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client['twitterdb']
tweet_collection = db['tweets']
app.static_folder = 'static'

# Function to fetch top 20 users
def get_top_users():
    top_users = tweet_collection.aggregate([
        {"$group": {"_id": "$user", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 20}
    ])
    return list(top_users)

@app.route('/api/tweet_distribution', methods=['GET'])
def get_tweet_distribution():
    # Get user query from the URL parameter
    user_query = request.args.get('user_query', '')

    # Build the filter based on the user query for the 'text' field
    filter_query = {}
    if user_query:
        # Construct the filter query for case-insensitive text search and existing date field
        filter_query = {
            "text": {"$regex": user_query, "$options": "i"},
            "date": {"$exists": True, "$type": "date"}
        }

    # Aggregate tweets day-wise
    tweet_distribution = tweets_collection.aggregate([
        {"$match": filter_query},  # Apply the filter
        {"$group": {
            "_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}},
            "count": {"$sum": 1}
        }},
        {"$sort": {"_id": 1}}
    ])

    # List of dictionaries where each dictionary represents a data point
    # with "date" and "count" fields.
    data = [{"date": entry["_id"], "count": entry["count"]} for entry in tweet_distribution]

    return jsonify(data)



if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
